In [1]:
import pandas as pd
import numpy as np
import math
import random


In [2]:
master_data = pd.read_csv("../../Data/filtered/master-data.csv", sep=",", encoding='Latin-1', low_memory=False)

In [3]:
master_data.head()

,compensation,year,country,age,gender,IT_experience_in_years,company_size,occupation,proficient_languages,desktop_OS,job_satisfaction
0,Student / Unemployed,2011,Africa,< 20,Female,<2,Start Up (1-25),Web Application Developer,['JavaScript'],Linux,FML
1,NaN,2011,Other Europe,25-29,Male,41310,Mature Small Business (25-100),Server Programmer,"['SQL', 'C']",Windows 7,So happy it hurts
2,NaN,2011,India,25-29,Female,41435,Mid Sized (100-999),Server Programmer,"['JavaScript', 'SQL']",Linux,NaN
3,Student / Unemployed,2011,Germany,< 20,Female,41310,Student,Student,['Haskell'],Linux,I enjoy going to work
4,"$80,000 - $100,000",2011,Other Asia,35-39,Male,11,Start Up (1-25),"Executive (VP of Eng, CTO, CIO, etc.)","['JavaScript', 'CSS', 'PHP', 'SQL', 'C++', 'C'...",Linux,It pays the bills


In [7]:
list(master_data["company_size"].unique())

['Start Up (1-25)',
 'Mature Small Business (25-100)',
 'Mid Sized (100-999)',
 'Student',
 'Fortune 1000 (1,000+)',
 nan,
 'Other (not working, consultant, etc.)',
 '101-999',
 '26-100',
 '1,000-3,000',
 '1/25/2013',
 '3001',
 '100',
 '1/5/2014',
 '50-100',
 '6/15/2014',
 '16-30',
 '31-50',
 'unknown',
 '100-499 employees',
 'I am not part of a company',
 '10-19 employees',
 '5-9 employees',
 '20-99 employees',
 '1-4 employees',
 '500-999 employees',
 '1,000-4,999 employees',
 '10,000+ employees',
 '5,000-9,999 employees',
 'I am not sure',
 'Other (please specify)',
 '20 to 99 employees',
 '10,000 or more employees',
 '10 to 19 employees',
 'Fewer than 10 employees',
 '5,000 to 9,999 employees',
 '100 to 499 employees',
 '1,000 to 4,999 employees',
 '500 to 999 employees',
 "I don't know",
 'I prefer not to answer',
 'Just me - I am a freelancer, sole proprietor, etc.',
 '2-9 employees',
 '2 to 9 employees',
 'I donÃ\x83Â¢Ã\x82Â\x80Ã\x82Â\x99t know']